In [474]:
import numpy as np
# size_x = 4
# x = np.random.rand(size_x)
# X = np.fft.fft(x)
# np.set_printoptions(precision=2)
# print(X)
# X_rev = X[::-1]
# X_rev = np.roll(X_rev, 1)
# print(X_rev)

x = np.random.rand(4,4)
X = np.fft.fft2(x)
print(X)

X_rev = X[::-1,::-1]
X_rev = np.roll(X_rev, 1, axis=0)
X_rev = np.roll(X_rev, 1, axis=1)
print(X_rev)


[[ 8.34+0.j    0.18-0.1j  -0.42+0.j    0.18+0.1j ]
 [ 0.1 +1.13j  0.05+1.39j  0.72+0.18j -0.77-1.3j ]
 [ 0.88+0.j   -0.33-0.15j  2.22+0.j   -0.33+0.15j]
 [ 0.1 -1.13j -0.77+1.3j   0.72-0.18j  0.05-1.39j]]
[[ 8.34+0.j    0.18+0.1j  -0.42+0.j    0.18-0.1j ]
 [ 0.1 -1.13j  0.05-1.39j  0.72-0.18j -0.77+1.3j ]
 [ 0.88+0.j   -0.33+0.15j  2.22+0.j   -0.33-0.15j]
 [ 0.1 +1.13j -0.77-1.3j   0.72+0.18j  0.05+1.39j]]


In [475]:
import numpy as np

size_x = 1<<2
size_y = 1<<2
x = np.random.rand(size_x, size_y).astype(np.float32)
y = np.random.rand(size_x, size_y).astype(np.float32)

fft_res = np.fft.fft2(x + 1j*y)

rev_res = fft_res[::-1, ::-1] 
rev_res = np.roll(rev_res, 1, axis=0)
rev_res = np.roll(rev_res, 1, axis=1)

#maintian 2 digit precision
np.set_printoptions(precision=1)
print(fft_res)
print(rev_res)

X = 1/2 * (fft_res + np.conj(rev_res))
Y = -1j/2 * (fft_res - np.conj(rev_res))

ifft_res = np.fft.ifft2(X + 1j*Y)

x_ifft = ifft_res.real
y_ifft = ifft_res.imag

print('max error:')
print(np.max(np.abs(x - x_ifft)))
print(np.max(np.abs(y - y_ifft)))

X_s_fft = np.fft.fft2(x)
Y_s_fft = np.fft.fft2(y)

print('max error:')
print(np.max(np.abs(X - X_s_fft)))
print(np.max(np.abs(Y - Y_s_fft)))

# sperate ifft
x_s_ifft = np.fft.ifft2(X)
y_s_ifft = np.fft.ifft2(Y)
x_ss_ifft = np.fft.ifft2(X_s_fft)
y_ss_ifft = np.fft.ifft2(Y_s_fft)

print('max error:')
print(np.max(np.abs(x - x_s_ifft.real)))
print(np.max(np.abs(y - y_s_ifft.real)))

print('max error:')
print(np.max(np.abs(x_ifft - x_ss_ifft.real)))
print(np.max(np.abs(y_ifft - y_ss_ifft.real)))

[[ 7.5+6.3j  2.1-0.4j -0.4+0.2j  0.8-0.6j]
 [-0.8+2.1j  1.2+1.4j -0.9-0.1j -0.3+0.2j]
 [ 0.9+0.1j -0.5+0.9j -1.3-2.2j -2.5+1.3j]
 [-2.5+0.1j  0.8+0.9j -0.9+1.4j -0.6-1.2j]]
[[ 7.5+6.3j  0.8-0.6j -0.4+0.2j  2.1-0.4j]
 [-2.5+0.1j -0.6-1.2j -0.9+1.4j  0.8+0.9j]
 [ 0.9+0.1j -2.5+1.3j -1.3-2.2j -0.5+0.9j]
 [-0.8+2.1j -0.3+0.2j -0.9-0.1j  1.2+1.4j]]
max error:
0.0
0.0
max error:
0.0
0.0
max error:
0.0
0.0
max error:
0.0
0.0


In [366]:
#1x2 对称复数
rand_complex = lambda: np.random.rand(1) + 1j*np.random.rand(1)
X1 = rand_complex()
X2 = X1.conj()
Y1 = rand_complex()
Y2 = Y1.conj()

print(np.array([X1,Y1]).T[0])

Z = np.array([X1, X2]) + 1j*np.array([Y1, Y2])

print(Z.T[0])

x_ = 1/2 * (Z[0] + np.conj(Z[1]))
y_ = -1/(2j) * (Z[0] - np.conj(Z[1]))
print(x_.T[0])
print(y_.T[0])

[0.1+0.5j 1. +0.3j]
[-0.2+1.5j  0.4+0.4j]
(0.07147644796086317+0.5425393977580595j)
(-0.9559459286243932-0.2856639832920498j)


In [9]:
print(-1j/2)

(-0-0.5j)


In [12]:
import cv2
import taichi as ti

ti.init(arch=ti.gpu)

img_cv = cv2.imread(
    r'C:/Users/Estelle/source/repos/TaichiSandBox/src/fft/ayanami.png', cv2.IMREAD_COLOR)
img_cv = cv2.resize(img_cv, (size_x, size_y))
img_cv = img_cv.astype(np.float32)
img_cv = img_cv / 255.0
# reorganize the image as rgb and inverse the y axis
img_cv = img_cv[::-1, :, ::-1]
# add alpha channel
img_cv = np.concatenate(
    (img_cv, np.ones((size_y, size_x, 1), dtype=np.float32)), axis=2)

# load to ti
rgba = ti.Vector.field(4, dtype=ti.f32, shape=(size_y, size_x))
rgba_freq = ti.Vector.field(4, dtype=ti.f32, shape=(size_y, size_x, 2))
# min_max_pixel = ti.Vector.field(3, dtype=ti.f32, shape=(size_y, size_x))
original = ti.Vector.field(4, dtype=ti.f32, shape=(size_y, size_x))

original.from_numpy(img_cv)
rgba.from_numpy(img_cv)

[Taichi] Starting on arch=cuda


In [9]:
import numpy as np
N = 16
M = 4

L = int(np.log(N)/np.log(M))

for l in range(1, L+1):
    for k in range(N):
        _l_ = L - l
        M_l = int(np.power(M, _l_))
        src_idx = [M*k + m*M_l - k % M_l for m in range(M)]
        print(f'l = {l}, k = {k}, src_idx = {src_idx}')
    print('------------------')

l = 1, k = 0, src_idx = [0, 4, 8, 12]
l = 1, k = 1, src_idx = [3, 7, 11, 15]
l = 1, k = 2, src_idx = [6, 10, 14, 18]
l = 1, k = 3, src_idx = [9, 13, 17, 21]
l = 1, k = 4, src_idx = [16, 20, 24, 28]
l = 1, k = 5, src_idx = [19, 23, 27, 31]
l = 1, k = 6, src_idx = [22, 26, 30, 34]
l = 1, k = 7, src_idx = [25, 29, 33, 37]
l = 1, k = 8, src_idx = [32, 36, 40, 44]
l = 1, k = 9, src_idx = [35, 39, 43, 47]
l = 1, k = 10, src_idx = [38, 42, 46, 50]
l = 1, k = 11, src_idx = [41, 45, 49, 53]
l = 1, k = 12, src_idx = [48, 52, 56, 60]
l = 1, k = 13, src_idx = [51, 55, 59, 63]
l = 1, k = 14, src_idx = [54, 58, 62, 66]
l = 1, k = 15, src_idx = [57, 61, 65, 69]
------------------
l = 2, k = 0, src_idx = [0, 1, 2, 3]
l = 2, k = 1, src_idx = [4, 5, 6, 7]
l = 2, k = 2, src_idx = [8, 9, 10, 11]
l = 2, k = 3, src_idx = [12, 13, 14, 15]
l = 2, k = 4, src_idx = [16, 17, 18, 19]
l = 2, k = 5, src_idx = [20, 21, 22, 23]
l = 2, k = 6, src_idx = [24, 25, 26, 27]
l = 2, k = 7, src_idx = [28, 29, 30, 31]
l = 2, k